### **Importing Required Libraries**

In [207]:
import pandas as pd
import matplotlib as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
import os

### **Loading CSV Data and Creating Data Frame**

In [208]:
data = pd.read_csv(r'C:\Users\njain\OneDrive - Cal State Fullerton\SPRING 2024\CPSC 597 Project\Project\APMBSS\data\processed\weather data\weather_dataset_train.csv')

In [209]:
data.sample(10)

,is_day,cloud_cover,precipitation,mood
4121,0,56.801865,1.981382,Sad
2464,1,43.361856,0.133802,Happy
879,0,12.960592,0.089157,Energetic
2179,0,47.487927,0.145680,Happy
412,0,6.235570,0.010319,Energetic
2509,0,28.786863,0.395687,Calm
710,0,9.209883,0.147115,Energetic
2678,0,0.190929,0.487311,Calm
1407,1,25.065198,0.101192,Happy
422,1,13.170898,0.094090,Energetic


### **Vectorizing Data Using Map Function**

In [210]:
dict_mood = {
    'Energetic': 1,
    'Happy': 2,
    'Calm': 3,
    'Sad': 4
}

data['encoded_mood'] = data['mood'].map(dict_mood)
data.sample(10)

,is_day,cloud_cover,precipitation,mood,encoded_mood
2609,0,28.639973,0.385141,Calm,3
2298,0,16.757704,0.182114,Happy,2
579,0,10.509650,0.107789,Energetic,1
509,1,23.812753,0.128759,Energetic,1
2306,1,32.813840,0.135517,Happy,2
2787,0,51.350536,0.057592,Calm,3
3981,1,31.981114,1.648457,Sad,4
1215,1,16.883581,0.014881,Energetic,1
2462,0,27.855663,0.102121,Happy,2
1449,1,27.015666,0.010466,Happy,2


In [211]:
# Dividing data frame into features and target on which the model will fit
features = ['is_day', 'cloud_cover', 'precipitation']
target = ['encoded_mood']

X = data[features]
y = data[target]

In [212]:
X.sample(10)

,is_day,cloud_cover,precipitation
3331,1,3.873215,0.397833
4628,0,29.827402,0.900220
108,0,4.273679,0.065832
4726,1,32.071980,1.635750
3801,1,12.984658,1.708395
2166,1,10.038661,0.202719
2815,0,47.935933,0.339103
1817,1,28.522007,0.110447
1545,1,25.038575,0.190665
2818,0,41.809391,0.539471


In [213]:
y.sample(10)

,encoded_mood
3895,4
3914,4
3863,4
1137,1
4818,4
2002,2
2329,2
2418,2
4091,4
2124,2


### **Vectorizing Test Data Using Map Function and Dividing into X_train and y_test**

In [214]:
# Loading test dataset using pandas
test_data = pd.read_csv(r'C:\Users\njain\OneDrive - Cal State Fullerton\SPRING 2024\CPSC 597 Project\Project\APMBSS\data\processed\weather data\weather_dataset_test.csv')

# Encoding mood using map() function to vectorize
test_data['encoded_mood'] = test_data['mood'].map(dict_mood)

X_test = test_data.drop('encoded_mood', axis=1)
X_test = X_test.drop('mood', axis=1)

In [215]:
# Viewing test data
X_test.sample(10)

,is_day,cloud_cover,precipitation
1478,0,3.663053,0.531155
195,0,1.353741,0.148610
690,0,28.809099,0.031036
1089,1,21.489153,0.234124
99,1,8.571154,0.039182
39,0,20.840864,0.040827
1012,1,49.554013,0.574525
585,1,45.760146,0.057246
1482,1,25.849637,0.592504
23,0,16.134401,0.088126


In [216]:
y_test = test_data['encoded_mood']

In [217]:
# Viewing the y_test
y_test

0       1
1       1
2       1
3       1
4       1
       ..
1995    4
1996    4
1997    4
1998    4
1999    4
Name: encoded_mood, Length: 2000, dtype: int64

### **Creating and Training the RandomForestClassifier**

In [218]:
# Creating random forest classifier object
rf = RandomForestClassifier(n_estimators=20, max_depth=10)

In [219]:
# Fitting the training data on random forest classifier
rf.fit(X, y)

c:\Users\njain\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=10, n_estimators=20)

### **Predicting the Accuracy**

In [220]:
y_pred = rf.predict(X_test)

In [221]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

In [222]:
accuracy

0.997

### **Predicting Mood on Random Value**

In [223]:
# 'is_day, cloud_cover, precipitation'
test_random_value = [0, 50, 0.0]

def get_key_from_value(dict_mood, predicted_mood_EncodedValue):
    for key, value in dict_mood.items():
        if value == predicted_mood_EncodedValue:
            return key
    # If the value is not found, you can handle it accordingly
    return None

predicted_mood_EncodedValue = rf.predict([test_random_value])[0]
predicted_mood = get_key_from_value(dict_mood, predicted_mood_EncodedValue)
print(predicted_mood)

Happy


c:\Users\njain\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


### **Saving the Mood Prediction Model from Weather**

In [224]:
# specific path for the model
model_directory = r'C:\Users\njain\OneDrive - Cal State Fullerton\SPRING 2024\CPSC 597 Project\Project\APMBSS\data\models'
model_filename = 'mood_detection_weather_model.pkl'
model_path = os.path.join(model_directory, model_filename)

# Save the model to the specified path
with open(model_path, 'wb') as file:
    pickle.dump(rf, file)